In [32]:
from google.cloud import bigquery
from google.oauth2 import service_account


In [4]:
key_path = "/Users/yco/.dbt/dbt-user-creds.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path#, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [66]:

from textacy import preprocessing

import html
import re

def replace_md_url(text):
    return re.compile(
        r"\[(.*)\]\(([^\(\)\[\]]*)\)"
    ).sub(r'\g<1> \g<2>', text)

preproc_pipe = preprocessing.make_pipeline(
    html.unescape,
    html.unescape,
    preprocessing.normalize.whitespace,
    preprocessing.normalize.bullet_points,
    preprocessing.normalize.hyphenated_words,
    replace_md_url,
    preprocessing.replace.urls,
    preprocessing.remove.brackets,
    preprocessing.normalize.unicode,
    preprocessing.remove.accents,
    preprocessing.replace.emojis,
    preprocessing.replace.numbers,
    preprocessing.replace.user_handles,
)

In [68]:
query = """
SELECT * from `reddit_texts.post_contents` LIMIT 20
"""
query_job = client.query(query)


rows = []
for i, row in enumerate(query_job):
    row = dict(row)
    text = row.pop("selftext")
    row["text"] = preproc_pipe(text)
    rows.append(row)

rows

[{'id': 'ruvj9n',
  'subreddit': 'datasets',
  'tag': 'API',
  'text': "I'm making an ESG stock analysis program in Java, and so far the only free ESG API I've come across is ESGEnterprise, but I'm having trouble retrieving the data. Has anyone had any success/have any recs for other ESG APIs out there."},
 {'id': 'rup1uj',
  'subreddit': 'datasets',
  'tag': 'API',
  'text': 'Hey everyone! I’m one of the creators of Sieve _URL_ and I’m excited to be sharing it!\nSieve _URL_ **is an API that helps you turn petabyte-scale video data into a high-quality dataset, automatically.**\nIt helps store, process, and semantically search your video data. Just think _NUMBER_ cameras recording footage at _NUMBER_ FPS, _NUMBER_/_NUMBER_. That would be _NUMBER_ million frames generated in a single day. The videos might be searchable by timestamp, but finding moments of interest is like searching for a needle in a haystack. Sieve tags useful attributes like people, motion, lighting, etc on every frame!